In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy
import tensorflow as tfl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import json

In [3]:
df = pd.read_csv('stock_nifty50.csv')

# GSPREAD connect model to google sheets

In [5]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [6]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
account = ServiceAccountCredentials.from_json_keyfile_name("rugged-matrix-378515-f839a205dadb.json",scope)
access = gspread.authorize(account)

# Dropdown for Stock Price Predictions

In [15]:
#stock=[]
#for i in df['Symbol'].unique():
#    stock.append({"userEnteredValue" : i})

In [17]:
# MAIN SPREADSHEET
Project = access.open_by_url('https://docs.google.com/spreadsheets/d/1-JNf_dPkk_RTPF5CGeF8sieuGFzbS3mdten5e0QNlzw/edit#gid=0')

In [ ]:
# sheetName = "Sheet1" # Sheet_Name.
# sheetId = Project.worksheet(sheetName).id # Extracting Sheet_ID
# body = {
#    "requests": [
#         {
#             "updateCells": {
#                 "range": {
#                     "sheetId": sheetId,
#                     "startRowIndex": 3,
#                     "endRowIndex": 4,
#                     "startColumnIndex": 1,
#                     "endColumnIndex": 2
#                 },
#                 "rows": [
#                     {
#                         "values": [
#                             {
#                                 "dataValidation": {
#                                     "condition": {
#                                         "values": stock,
#                                         "type": "ONE_OF_LIST"
#                                     },
#                                     "showCustomUi": True
#                                 }
#                             }
#                         ]
#                     }
#                 ],
#                 "fields": "dataValidation"
#             }
#         }
#     ]
#  }
# Project.batch_update(body)

In [18]:
NSE50 = Project.get_worksheet(0)
NSE50

<Worksheet 'Sheet1' id:0>

In [19]:
data = pd.DataFrame(NSE50.get_all_records())

# Stock Prediction LSTM Model

In [29]:
def stockPredictionModel(name_of_stock, number_of_share, start_date, end_date):
    df = pd.read_csv('stock_nifty50.csv')
    df.head()
    df["Date"] = pd.to_datetime(df["Date"])
    # name_of_stock = input("Enter Name Of Stock")
    # number_of_share = int(input("Number Of Share"))
    Y = df[df["Symbol"]==name_of_stock][["Close"]]
    # start_date = input("Start Date")
    # end_date = input("End Date")
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)



    scaler=MinMaxScaler(feature_range=(0,1))
    Y=scaler.fit_transform(np.array(Y).reshape(-1,1))



    training_size=int(len(Y)*0.65)
    test_size=len(Y)-training_size
    train_data,test_data=Y[0:training_size,:],Y[training_size:len(Y),:1]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
            dataX.append(a)
            dataY.append(dataset[i + time_step, 0])
        return numpy.array(dataX), numpy.array(dataY)


    time_step = 100
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, ytest = create_dataset(test_data, time_step)


    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

    ### Create the Stacked LSTM model
    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=5,batch_size=64,verbose=1)

    last_date = pd.to_datetime("2022-12-12")
    lst_output_1=[]
    lst_output_2=[]

    diff_1 = start_date-last_date
    diff_2 = end_date-last_date
    if int(diff_1.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(diff_1.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                lst_output_1.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                lst_output_1.extend(yhat.tolist())
                i=i+1


            
    if int(diff_2.days)>0:
        n_steps=100
        x_input=test_data[len(test_data)-n_steps:].reshape(1,-1)
        temp_input=list(x_input)
        temp_input=temp_input[0].tolist()
        i=0
        while(i<int(diff_2.days)):
            if(len(temp_input)>100):
                #print(temp_input)
                x_input=np.array(temp_input[1:])
    #                 print("{} day input {}".format(i,x_input))
                x_input=x_input.reshape(1,-1)
                x_input = x_input.reshape((1, n_steps, 1))
                #print(x_input)
                yhat = model.predict(x_input, verbose=0)
    #                 print("{} day output {}".format(i,yhat))
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                #print(temp_input)
                lst_output_2.extend(yhat.tolist())
                i=i+1
            else:
                x_input = x_input.reshape((1, n_steps,1))
                yhat = model.predict(x_input, verbose=0)
    #                 print(yhat[0])
                temp_input.extend(yhat[0].tolist())
    #                 print(len(temp_input))
                lst_output_2.extend(yhat.tolist())
                i=i+1
    if int(diff_2.days)>0 and int(diff_1.days)>0:     
      invests = scaler.inverse_transform(lst_output_1)[-1]
      returns = scaler.inverse_transform(lst_output_2)[-1]
      Final_Amount = ((returns-invests)*(int(number_of_share)))
    elif int(diff_2.days)>0 and int(diff_1.days)<0: 
      invests = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == start_date)]['Close'].values[0]).reshape(1,-1)
      invests= invests[-1] * (int(number_of_share))
      returns = (scaler.inverse_transform(lst_output_2)[-1])* (int(number_of_share))
      Final_Amount = (returns-invests)
    else:
      invests = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == start_date)]['Close'].values[0]).reshape(1,-1)
      invests= invests[-1] * (int(number_of_share))
      returns = (df[(df['Symbol'] == name_of_stock) & (df['Date'] == end_date)]['Close'].values[0]).reshape(1,-1)
      returns= returns[-1] * (int(number_of_share))
      
      Final_Amount = (returns-invests)


    # In[2]:

    if Final_Amount[0]>0:
        print("Profit: " ,Final_Amount[0])
        NSE50.update('B9', Final_Amount[0])
        return Final_Amount[0]
    else:
        print("Loss: " ,Final_Amount[0])
        NSE50.update('B9', Final_Amount[0])
        return Final_Amount[0]

In [30]:
stockPredictionModel('ADANIENT',30,'2023-10-1', '2023-12-12')

Epoch 1/5
9/9 [==============================] - 10s 315ms/step - loss: 0.0069 - val_loss: 0.0080
Epoch 2/5
9/9 [==============================] - 1s 150ms/step - loss: 0.0011 - val_loss: 0.0028
Epoch 3/5
9/9 [==============================] - 1s 157ms/step - loss: 6.3819e-04 - val_loss: 0.0034
Epoch 4/5
9/9 [==============================] - 1s 160ms/step - loss: 3.4548e-04 - val_loss: 0.0032
Epoch 5/5
9/9 [==============================] - 2s 181ms/step - loss: 2.7106e-04 - val_loss: 0.0050
Loss:  -190.86932498216356


-190.86932498216356

In [32]:
stockPredictionModel('WIPRO', 30,'2020-2-25', '2022-11-30')

Epoch 1/5
9/9 [==============================] - 11s 453ms/step - loss: 0.0407 - val_loss: 0.0157
Epoch 2/5
9/9 [==============================] - 2s 221ms/step - loss: 0.0085 - val_loss: 0.0156
Epoch 3/5
9/9 [==============================] - 2s 230ms/step - loss: 0.0040 - val_loss: 0.0049
Epoch 4/5
9/9 [==============================] - 2s 220ms/step - loss: 0.0017 - val_loss: 0.0063
Epoch 5/5
9/9 [==============================] - 2s 215ms/step - loss: 0.0014 - val_loss: 0.0046
Profit:  4884.0


4884.0

In [31]:
stockPredictionModel('WIPRO', 30,'2023-1-31', '2025-12-12')

Epoch 1/5
9/9 [==============================] - 10s 358ms/step - loss: 0.0497 - val_loss: 0.0068
Epoch 2/5
9/9 [==============================] - 1s 168ms/step - loss: 0.0096 - val_loss: 0.0138
Epoch 3/5
9/9 [==============================] - 2s 176ms/step - loss: 0.0040 - val_loss: 0.0041
Epoch 4/5
9/9 [==============================] - 2s 170ms/step - loss: 0.0023 - val_loss: 0.0098
Epoch 5/5
9/9 [==============================] - 2s 168ms/step - loss: 0.0016 - val_loss: 0.0045
Loss:  -6731.636626878754


-6731.636626878754

# Stock Price Prediction

In [33]:
while(True):
    name_of_stock = NSE50.acell('B4').value
    number_of_share = NSE50.acell('B5').value
    start_date = NSE50.acell('B6').value
    end_date = NSE50.acell('B7').value

    stockPredictionModel(name_of_stock, number_of_share, start_date, end_date)
    time.sleep(5)


Epoch 1/5
9/9 [==============================] - 12s 419ms/step - loss: 0.0453 - val_loss: 0.0402
Epoch 2/5
9/9 [==============================] - 2s 230ms/step - loss: 0.0071 - val_loss: 0.0542
Epoch 3/5
9/9 [==============================] - 2s 230ms/step - loss: 0.0043 - val_loss: 0.0369
Epoch 4/5
9/9 [==============================] - 2s 233ms/step - loss: 0.0039 - val_loss: 0.0420
Epoch 5/5
9/9 [==============================] - 2s 226ms/step - loss: 0.0030 - val_loss: 0.0321
Loss:  -341000.8702656626
Epoch 1/5
9/9 [==============================] - 12s 507ms/step - loss: 0.0517 - val_loss: 0.0385
Epoch 2/5
9/9 [==============================] - 2s 219ms/step - loss: 0.0083 - val_loss: 0.0470
Epoch 3/5
9/9 [==============================] - 2s 220ms/step - loss: 0.0046 - val_loss: 0.0370
Epoch 4/5
9/9 [==============================] - 2s 210ms/step - loss: 0.0035 - val_loss: 0.0382
Epoch 5/5
9/9 [==============================] - 2s 211ms/step - loss: 0.0029 - val_loss: 0.0325
Lo

KeyboardInterrupt: 